In [1]:
# imports 

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

import statsmodels.api as sm

In [3]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/project-2/code'

In [4]:
cd ..

/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/project-2


In [5]:
# loading in datasets 

train = pd.read_csv('./datasets/train.csv')
test = pd.read_csv('./datasets/test.csv')

**Data Dictionary:** 
https://www.kaggle.com/c/dsir-1116-ames-regression-challenge/data

In [6]:
# checking out data

train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [23]:
len(train)

2051

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

In [9]:
train.isnull().sum()

Id                0
PID               0
MS SubClass       0
MS Zoning         0
Lot Frontage    330
               ... 
Misc Val          0
Mo Sold           0
Yr Sold           0
Sale Type         0
SalePrice         0
Length: 81, dtype: int64

In [20]:
nulls = pd.DataFrame(train.isnull().sum())
nulls

,0
Id,0
PID,0
MS SubClass,0
MS Zoning,0
Lot Frontage,330
...,...
Misc Val,0
Mo Sold,0
Yr Sold,0
Sale Type,0


In [22]:
#count of null values for each feature with null values 
nulls[nulls[0]!=0]

,0
Lot Frontage,330
Alley,1911
Mas Vnr Type,22
Mas Vnr Area,22
Bsmt Qual,55
Bsmt Cond,55
Bsmt Exposure,58
BsmtFin Type 1,55
BsmtFin SF 1,1
BsmtFin Type 2,56


In [17]:
train.loc[:, train.isna().any()].T

,0,1,2,3,4,5,6,7,8,9,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
Lot Frontage,NaN,43,68,73,82,137,35,NaN,NaN,70,...,110,45,50,65,96,79,NaN,57,80,60
Alley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Grvl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mas Vnr Type,BrkFace,BrkFace,None,None,None,None,BrkFace,BrkFace,None,BrkFace,...,BrkFace,None,None,BrkFace,Stone,None,None,None,None,None
Mas Vnr Area,289,132,0,0,0,0,82,180,0,192,...,664,0,0,38,426,0,0,0,0,0
Bsmt Qual,TA,Gd,TA,Gd,Fa,TA,Gd,TA,TA,Gd,...,Ex,TA,TA,Gd,Ex,Gd,TA,TA,TA,TA
Bsmt Cond,TA,TA,TA,TA,Gd,TA,TA,TA,TA,TA,...,TA,TA,TA,TA,TA,TA,TA,TA,TA,TA
Bsmt Exposure,No,No,No,No,No,No,Gd,No,No,Av,...,Av,No,Mn,No,Av,Av,No,No,No,No
BsmtFin Type 1,GLQ,GLQ,GLQ,Unf,Unf,ALQ,GLQ,Rec,Rec,Rec,...,GLQ,Unf,Rec,GLQ,GLQ,GLQ,BLQ,Unf,Rec,Unf
BsmtFin SF 1,533,637,731,0,0,247,547,1000,292,650,...,1016,0,276,362,1336,1011,262,0,155,0
BsmtFin Type 2,Unf,Unf,Unf,Unf,Unf,Rec,Unf,Unf,Unf,Unf,...,Unf,Unf,Unf,Unf,Unf,Unf,Unf,Unf,LwQ,Unf


In [12]:
# showing only features that have null values 

train[train.isnull().sum()!=0]

<ipython-input-12-78acdaf2c97a>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train[train.isnull().sum()!=0]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).